1. Scrapy

Objectif : comprendre ce qu'est Scrapy et quand l'utiliser.

Scrapy est un framework Python dédié au scraping web. Il fournit :
- Une architecture prête à l'emploi (spiders, pipelines, middlewares).
- Un moteur asynchrone performant.
- Un système de requêtes/réponses simple à tester.

Quand l'utiliser :
- Pour crawler plusieurs pages avec un vrai suivi de liens.
- Pour industrialiser une collecte avec ré-essais et règles propres.
- Quand on veut séparer extraction, nettoyage et stockage.

Installation (hors notebook) :
```
pip install scrapy
```

In [ ]:
# Vérifie l'import de Scrapy (nécessite l'installation préalable).
import scrapy

# Affiche la version installée pour la traçabilité.
print('Scrapy version:', scrapy.__version__)

2. Introduction aux mécanismes de base du Framework

Objectif : parcourir les composants essentiels de Scrapy.

Composants principaux :
- Spider : définit comment crawler et extraire les données.
- Request/Response : encapsulent les requêtes et les réponses HTTP.
- Item : structure de données pour les champs extraits.
- Pipeline : nettoyage, validation et persistance.

Ci-dessous, on crée une Spider minimale qui parse un HTML local.
Cela permet d'expliquer le flux sans faire d'appel réseau réel.

In [ ]:
from scrapy import Spider
from scrapy.http import TextResponse, Request

# Définition d'un Item simple (structure des données extraites).
class ArticleItem(dict):
    # On utilise un dict pour rester simple dans un notebook.
    # En projet Scrapy, on utiliserait scrapy.Item + scrapy.Field.
    pass

# Spider minimaliste qui parse du HTML statique.
class DemoSpider(Spider):
    # Nom unique de la spider.
    name = 'demo'

    # Fonction de parsing appelée avec une Response.
    def parse(self, response):
        # Sélectionne les titres d'articles via un sélecteur CSS.
        for title in response.css('h2.article-title::text').getall():
            # Nettoie l'espace inutile autour du texte.
            cleaned = title.strip()
            # Construit l'item et le renvoie.
            item = ArticleItem(title=cleaned)
            yield item

# HTML local pour simuler une page web.
html = """
<html>
  <body>
    <h2 class="article-title"> Introduction à Scrapy </h2>
    <h2 class="article-title"> Crawler des liens proprement </h2>
  </body>
</html>
"""

# Fabrique un objet Response comme si Scrapy l'avait créé.
request = Request(url='https://example.local')
response = TextResponse(url='https://example.local', request=request, body=html, encoding='utf-8')

# Instancie la spider et exécute parse manuellement.
spider = DemoSpider()
items = list(spider.parse(response))

# Affiche les items extraits.
print(items)

2.bis. Anatomie d'un projet Scrapy (structure)

Objectif : comprendre les dossiers standards créés par scrapy startproject.

Structure typique :
- spiders/ : contient les spiders.
- items.py : définition des champs.
- pipelines.py : traitement et stockage.
- settings.py : configuration globale.

Dans un vrai projet, on lance :
```
scrapy startproject monprojet
scrapy genspider example example.com
```

2.ter. Exemple simple de Request et callback

Objectif : illustrer le cycle Request → Response → parse.

Ici, on ne fait pas d'appel réseau. On simule la réponse reçue.

In [ ]:
from scrapy.http import TextResponse

# URL cible fictive.
url = 'https://example.local/articles'

# HTML de réponse fictive.
html2 = """
<html>
  <body>
    <a href="/a1">Article 1</a>
    <a href="/a2">Article 2</a>
  </body>
</html>
"""

# Simule la réponse.
response2 = TextResponse(url=url, body=html2, encoding='utf-8')

# Extraction de liens par sélecteur CSS.
links = response2.css('a::attr(href)').getall()

# Affiche les liens extraits.
print('Liens:', links)

2.quater. Items et Pipelines (nettoyage et validation)

Objectif : montrer comment structurer les données et les nettoyer.

Idée :
- Un item est une structure de données normalisée.
- Un pipeline transforme l'item (nettoyage, filtrage, stockage).

Dans un vrai projet, ces classes vivent dans items.py et pipelines.py.

In [ ]:
# Définition d'un item explicite avec des champs connus.
class BlogPostItem(dict):
    # Ici on reste simple (dict). Scrapy propose scrapy.Item.
    pass

# Pipeline minimaliste pour nettoyer un titre.
class CleanTitlePipeline:
    def process_item(self, item, spider):
        # Récupère le champ title si présent.
        title = item.get('title', '')
        # Nettoie les espaces superflus.
        item['title'] = title.strip()
        # Filtre : si le titre est vide, on lève une erreur.
        if not item['title']:
            raise ValueError('Titre vide')
        # Retourne l'item nettoyé.
        return item

# Démonstration manuelle du pipeline.
raw_item = BlogPostItem(title='  Hello Scrapy  ')
pipeline = CleanTitlePipeline()
clean_item = pipeline.process_item(raw_item, spider=None)
print(clean_item)

2.quinquies. Exemple réseau réel (crawler simple)

Objectif : exécuter une vraie requête HTTP avec Scrapy.

Le spider ci-dessous récupère le titre de https://example.com/.
C'est un site public de test, stable et adapté aux démonstrations.

In [ ]:
from scrapy.crawler import CrawlerProcess
from scrapy import Request

# Spider minimal pour récupérer le titre d'une page réelle.
class ExampleComSpider(Spider):
    name = 'example_com'
    allowed_domains = ['example.com']
    start_urls = ['https://example.com/']

    def parse(self, response):
        # Extrait le contenu de la balise <title>.
        page_title = response.css('title::text').get()
        # Nettoie les espaces éventuels.
        page_title = (page_title or '').strip()
        # Retourne un item simple.
        yield {'title': page_title, 'url': response.url}

# Configuration minimale pour un run dans un notebook.
settings = {
    # Empêche Scrapy de créer des fichiers logs trop verbeux.
    'LOG_LEVEL': 'ERROR',
    # Respecte le robots.txt pour un comportement correct.
    'ROBOTSTXT_OBEY': True,
}

# Lance le crawler (cela peut prendre quelques secondes).
process = CrawlerProcess(settings=settings)
process.crawl(ExampleComSpider)
process.start()

2.sexies. Suivre des liens et normaliser les donnees

Objectif : montrer response.follow et un ItemLoader simple.

Idee :
- Extraire des liens depuis une page.
- Suivre chaque lien avec un callback.
- Nettoyer les champs via un loader.

Cet exemple reste local et ne lance pas de vrai crawl.

In [ ]:
from scrapy.loader import ItemLoader
from itemloaders.processors import TakeFirst, MapCompose

# Item explicite avec un loader pour normaliser.
class LinkItem(scrapy.Item):
    url = scrapy.Field()
    label = scrapy.Field()

# Loader qui applique des transformations simples.
class LinkItemLoader(ItemLoader):
    default_output_processor = TakeFirst()
    label_in = MapCompose(str.strip)

# HTML local avec des liens.
html3 = """
<html>
  <body>
    <a href="/a1">  Article 1  </a>
    <a href="/a2">  Article 2  </a>
  </body>
</html>
"""

# Simule une reponse.
response3 = TextResponse(url='https://example.local', body=html3, encoding='utf-8')

# Extrait chaque lien et construit un item.
items3 = []
for link in response3.css('a'):
    loader = LinkItemLoader(item=LinkItem(), selector=link)
    loader.add_css('url', '::attr(href)')
    loader.add_css('label', '::text')
    items3.append(loader.load_item())

print(items3)

2.septies. Export JSON (idee de pipeline)

Objectif : montrer comment ecrire des items en JSON.

Dans un vrai projet, on configurerait FEEDS dans settings.py.
Ici, on simule l'ecriture avec json pour la pedagogie.

In [ ]:
import json

# Items d'exemple a exporter.
sample_items = [
    {'title': 'Intro Scrapy', 'url': 'https://example.com/'},
    {'title': 'Crawler propre', 'url': 'https://example.com/2'},
]

# Ecrit un fichier JSON local.
with open('scrapy_items.json', 'w', encoding='utf-8') as f:
    json.dump(sample_items, f, ensure_ascii=False, indent=2)

print('Export JSON ok:', len(sample_items))

3. Exercices

1. Ajouter un champ "url" dans l'item renvoye par la spider.
2. Extraire un resume depuis un <p> et nettoyer les espaces.
3. Modifier la spider pour suivre des liens (yield Request).
4. Ajouter un pipeline qui filtre les titres vides.
5. Ecrire un spider qui scrape JSONPlaceholder (posts).
6. Enregistrer les items dans un fichier JSON.

Conseil : commencez par une page HTML locale pour valider vos selecteurs.
